<a href="https://colab.research.google.com/github/et169tkm/gpt35turbo_test_text_parsing/blob/main/ChatGPT_API_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports and functions { vertical-output: true }

!pip install openai >/dev/null

import bs4
import json
import numpy as np
import os
import openai
import requests

from bs4 import BeautifulSoup

API_KEY = input("API Key")
openai.api_key = API_KEY


# functions

# Recursively check if element "e" matches, perform fn() on it if it matches
def perform_on_type(e, matcher, fn):
  if matcher(e):
    fn(e)
  else:
    if getattr(e, "children", None): # check if it has the "children" attribute
      for child in e.children:
        perform_on_type(child, matcher, fn)


# scrap text from right move and minor preprocessing
def scrap_right_move(url):
  r = requests.get(url)
    
  # soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install 
  soup = BeautifulSoup(r.content, 'html.parser')


  # Find the titl element
  elements = soup.find_all("h1")
  title = None
  for e in elements:
    if e.get("itemprop") == "streetAddress":
      title = e.string

  # Find the element that contains the text
  elements = soup.find_all("article")
  data_element = None
  for e in elements:
    # print(e.get("data-testid"))
    # print(e.decode_contents())
    if e.get("data-testid") == "primary-layout":
      data_element = e
      break

  perform_on_type(
      data_element,
      lambda x: isinstance(x, bs4.element.NavigableString), # matcher
      lambda x: x.string.replace_with(x.string + " ")) # fn

  # Remove tooltips
  for e in data_element.find_all("span"):
    if e.get("role") == "tooltip":
      # tooltip_elements.append(e)
      e.clear()

  data = data_element.text
  data = data.replace("\r\n", " ")
  data = data.replace("\n", " ")
  data = data.replace("\r", " ")
  return (title, data)

In [ ]:
#@title extract function(gpt-3.5-turbo) { vertical-output: false }

# Prepare question

FIELDS = [
  "floor",
  "sqft",
  "leasehold/freehold/tenure",
  "lease",
  "EPC",
  "council tax",
  "service charge",
  "ground rent",
  "chain",
  "parking/garage",
  "lift"
]
PROMPT_FORMAT_1 = """
A table summerizing the following data (put "null" if the information is not available):

{data}

| """ + " | ".join(FIELDS) + " | "

def format_output(output):
  values = output.split("|")
  if len(values) == len(FIELDS) + 2:
    if values[0].strip() == "" and values[-1].strip() == "":
      values = values[1:-1]
  if len(values) != len(FIELDS):
    print("Incorrect number of fields: %d (expect: %d)" % (len(values), len(FIELDS)))
    print("Expected fields: %s" % "|".join(FIELDS))
    return

  out_format = "%% %ds: %%s" % np.max([len(x) for x in FIELDS])
  for i in range(len(FIELDS)):
    print(out_format % (FIELDS[i], values[i].strip()))

def extract_data(data):
  # Call ChatGPT API
  question = PROMPT_FORMAT_1.format(data=data)
  response_3_5_turbo = openai.ChatCompletion.create(   
    model="gpt-3.5-turbo",
    temperature=0.2, # (0 to 2) low value = more deteriministic
    # frequency_penalty=-1.8, # (-2 to 2) penalty for repeating words verbatim, in this case we do want it to repeat verbatim
    messages=[         
      {
        "role": "user",
        "content": question
      },
    ])
  response_string = response_3_5_turbo.choices[0].message.content
  format_output(response_string)
  print("Raw response: \"%s\"" % response_string)

In [ ]:
#@title main { vertical-output: false }


URL = "" #@param {type:"string"}

title, data = scrap_right_move(URL)
print("title: %s" % title)
print("data: %s" % data)
extract_data(data)

In [ ]:
#@title davinci { vertical-output: false }

response_davinci = openai.Completion.create(
  model="text-davinci-003",
  prompt=question,
  temperature=0,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
response_json = response_davinci.choices[0].text
print(response_json)